# FAST: Feedforward-ASsisted Transformers

Notebook for running GLUE tasks.

# Setup

## Modules

In [1]:
import random
import time
import csv
import os
import pathlib
import itertools
from datetime import datetime
from collections import namedtuple
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.preprocessing import OneHotEncoder

from utils.feed_forward import FeedForward
from utils.cls import extract_cls_embeddings
from utils.mean_pooling import mean_pooling
from utils.energy import get_energy
from utils.embeddings import add_embeddings

## Device

In [2]:
# Set Python environment variables for CUBLAS (NVIDIA)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ['CUDNN_DETERMINISTIC'] = '1'

# Standardized default seed
seed = 7
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(mode=True)

In [3]:
device_name = "cpu"  # Default device is CPU
if torch.cuda.is_available():
    # I read that this works for detecting if notebook is being run in a colab environment, not sure though
    if 'COLAB_GPU' in os.environ:
        print("colab environment")
        device_name = "gpu" 
    else:
        device_name = "cuda" # CUDA for NVIDIA GPU
elif torch.backends.mps.is_available():
    device_name = torch.device("mps")  # Metal Performance Shaders for Apple M-series GPU

# device_name = "cuda:0"
device = torch.device(device_name)
print(device_name)

mps


## User parameters

Parameters to set:
- Model
    - MPNet
    - DistilRoBERTa
- Task
    - cola
    - sst2
    - mrpc
    - stsb
    - qqp
    - mnli_matched
    - mnli_mismatched
    - qnli
    - rte
    - wnli
- Embedding type
    - Single Sentence
        - cls
        - meanpool
        - sentence
    - Two Sentence
        - Each sentence separately
            - cls_single
            - meanpool_single
        - Both sentences at once
            - cls_double
            - meanpool_double
        - sentence_single (no sentence_double option)

In [4]:
model_param = "DistilRoBERTa"
task_param = "wnli"
embedding_param = ["meanpool_single"]

## Task setup

In [5]:
if model_param == "MPNet":

    if "cls" in "_".join(embedding_param) or "meanpool" in "_".join(embedding_param):
        from transformers import MPNetTokenizer, MPNetModel
        tokenizer = MPNetTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
        model = MPNetModel.from_pretrained("microsoft/mpnet-base").to(device)
        
    if "sentence" in "_".join(embedding_param):
        from sentence_transformers import SentenceTransformer
        sentence_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

elif model_param == "DistilRoBERTa":

    if "cls" in "_".join(embedding_param) or "meanpool" in "_".join(embedding_param):
        from transformers import RobertaTokenizer, RobertaModel
        tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
        model = RobertaModel.from_pretrained('distilroberta-base').to(device)

    if "sentence" in "_".join(embedding_param):
        from sentence_transformers import SentenceTransformer
        sentence_model = SentenceTransformer('sentence-transformers/all-distilroberta-v1').to(device)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
TaskConfig = namedtuple("TaskConfig", ["sentence_type", "class_type", "num_classes", "col_names"])

task_configs = {
    "cola": TaskConfig("one", "BC", 1, ['sentence']),
    "sst2": TaskConfig("one", "BC", 1, ['sentence']),
    "mrpc": TaskConfig("two", "BC", 1, ['sentence1', 'sentence2']),
    "stsb": TaskConfig("two", "R", 1, ['sentence1', 'sentence2']),
    "qqp": TaskConfig("two", "BC", 1, ['question1', 'question2']),
    "mnli_matched": TaskConfig("two", "MC", 3, ['premise', 'hypothesis']),
    "mnli_mismatched": TaskConfig("two", "MC", 3, ['premise', 'hypothesis']),
    "qnli": TaskConfig("two", "BC", 1, ['question', 'sentence']),
    "rte": TaskConfig("two", "BC", 1, ['sentence1', 'sentence2']),
    "wnli": TaskConfig("two", "BC", 1, ['sentence1', 'sentence2']),
}

task_config = task_configs[task_param]

In [7]:
if task_param == "mnli_matched": 
    data = load_dataset("glue", "mnli") 
    val_key = "validation_matched"
    test_key = "test_matched"
elif task_param == "mnli_mismatched":
    data = load_dataset("glue", "mnli") 
    val_key = "validation_mismatched"
    test_key = "test_mismatched"
else:
    data = load_dataset("glue", task_param)
    val_key = "validation"
    test_key = "test"

data

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 635
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 71
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 146
    })
})

# Embeddings

## Obtain labels

In [8]:
Y_train = np.array(data["train"]["label"])
Y_val = np.array(data[val_key]["label"])
Y_test = np.array(data[test_key]["label"])

if task_config.class_type == "MC":
    Y_train = np.reshape(Y_train, (-1, 1))
    Y_val = np.reshape(Y_val, (-1, 1))
    Y_test = np.reshape(Y_test, (-1, 1))
    
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(Y_train)
    print(ohe.categories_)
     
    Y_train = ohe.transform(Y_train)
    Y_val = ohe.transform(Y_val)
    Y_test = ohe.transform(Y_test)

## Check for saved embeddings

In [9]:
total_length = len(embedding_param)

X_train, X_val, X_test, embedding_tracker = [None]*total_length, [None]*total_length, [None]*total_length, []
create_dataloader, create_sentence = False, False

for id, embedding in enumerate(embedding_param):
    
    cache_path = pathlib.Path(f"./cache/{embedding}/{task_param}")
    cache_path.mkdir(parents=True, exist_ok=True)

    file_names = ['X_train', 'X_val', 'X_test'] 
    paths = [pathlib.Path(cache_path / f"{f}_{model_param}.npy") for f in file_names]

    if all(path.exists() for path in paths):
        print(f"{embedding} embeddings found!")
        X_train[id] = np.load(paths[0])
        X_val[id] = np.load(paths[1])
        X_test[id] = np.load(paths[2])

        print(f"X_train shape: {X_train[0].shape}")
        print(f"X_val shape  : {X_val[0].shape}")
        print(f"X_test shape : {X_test[0].shape}")

    else:
        embedding_tracker.append(id)
        print(f"No {embedding} saved embeddings found")

        if "cls" in embedding_param[id] or "meanpool" in embedding_param[id]:
            create_dataloader = True
        elif "sentence" in embedding_param[id]:
            create_sentence = True

meanpool_single embeddings found!
X_train shape: (635, 1536)
X_val shape  : (71, 1536)
X_test shape : (146, 1536)


## Tokenize dataset

In [10]:
# Define tokenization functions
max_len=512

def tokenize(examples, name):
    tokenized = tokenizer(examples[task_config.col_names[0]],
                          add_special_tokens=True,
                          padding='max_length',
                          truncation=True,
                          max_length=max_len,
                          return_tensors='pt')
    return {name + "_" + key: value.to(device) for key, value in tokenized.items()}

def tokenize_single(examples, name):
    tokenized_1 = tokenizer(examples[task_config.col_names[0]],
                         add_special_tokens=True,
                         padding='max_length',
                         truncation=True,
                         max_length=max_len,
                         return_tensors='pt')
    tokenized_2 = tokenizer(examples[task_config.col_names[1]],
                         add_special_tokens=True,
                         padding='max_length',
                         truncation=True,
                         max_length=max_len,
                         return_tensors='pt')
    
    tokenized_output = {name + "_" + key + "_1": value.to(device) for key, value in tokenized_1.items()}
    tokenized_output.update({name + "_" + key + "_2": value.to(device) for key, value in tokenized_2.items()})
    return tokenized_output

def tokenize_double(examples, name):
    tokenized = tokenizer(examples[task_config.col_names[0]],
                          examples[task_config.col_names[1]],
                          add_special_tokens=True,
                          padding='max_length',
                          truncation=True,
                          max_length=max_len,
                          return_tensors='pt')
    return {name + "_" + key: value.to(device) for key, value in tokenized.items()}

In [11]:
inputs = []
for id, embedding in enumerate(embedding_param):
    if id in embedding_tracker and "sentence" not in embedding:
        if "single" in embedding:
            inputs.append(data.map(lambda examples: tokenize_single(examples, name=embedding), batched=True))
        elif "double" in embedding:
            inputs.append(data.map(lambda examples: tokenize_double(examples, name=embedding), batched=True))
        else:
            inputs.append(data.map(lambda examples: tokenize(examples, name=embedding), batched=True))

## Create Dataloader

In [12]:
# Define batch size
batch_size = 128

if create_dataloader:
    from utils.dataloader import TransformerDataset, clean_dataset
    
    # Merge datasets for each split and convert to PyTorch datasets
    train_df = clean_dataset(inputs, 'train')
    validation_df = clean_dataset(inputs, 'validation')
    test_df = clean_dataset(inputs, 'test')

    # Create dataset instances
    train_dataset = TransformerDataset(train_df)
    validation_dataset = TransformerDataset(validation_df)
    test_dataset = TransformerDataset(test_df)

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Compute Embbeddings

In [13]:
# Define embedding/sentence functions
def compute_embeddings(loader, X):
    column_names = list(next(iter(loader)).keys())

    # Creating pairs of input IDs and attention masks
    pairs = []
    for name in column_names:
        if 'input_ids' in name:
            mask_name = name.replace('input_ids', 'attention_mask')
            pairs.append((name, mask_name))

    # Loop over each pair of input ID and attention mask
    for input_id_name, mask_name in pairs:
        embedding = []
        tqdm.write(f"{input_id_name} {mask_name}")
        # time.sleep(0.2) # tqdm prints weird without slight time delay (will make embedding time inaccurate)
        
        for input_embedding in tqdm(loader):
            # Move batch to device
            model.eval()
            input_embedding = {key: value.to(device) for key, value in input_embedding.items()}
            with torch.no_grad():
                outputs = model(input_embedding[input_id_name], attention_mask=input_embedding[mask_name])

                if "cls" in input_id_name:
                    embed = extract_cls_embeddings(outputs)
                elif "meanpool" in input_id_name:
                    embed = mean_pooling(outputs, input_embedding[mask_name])

            embedding.append(embed.cpu().numpy())

        id = embedding_param.index(input_id_name.split("_input_ids")[0])
        # print(id)
        if "2" in input_id_name:
            U, V = X[id], np.concatenate(embedding, axis=0)
            X[id] = np.hstack([U, V])
            # print(X[id])
        else:
            X[id] = np.concatenate(embedding, axis=0)
            # print(X[id])

def compute_sentence(sentences):
    embeddings = []
    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i:i + batch_size]
        batch_embeddings = sentence_model.encode(batch_sentences)
        embeddings.append(batch_embeddings)
    return np.concatenate(embeddings, axis=0)

In [14]:
print("Embedding: Using default time tracking")
# Track time
start_time = time.time()

if create_dataloader: # Compute CLS/Meanpool embeddings
    compute_embeddings(train_loader, X_train)
    compute_embeddings(validation_loader, X_val)
    compute_embeddings(test_loader, X_test)
if create_sentence: # Compute Sentence embeddings
    if "sentence_single" in embedding_param:
        id = embedding_param.index("sentence_single")
        U_train, V_train = compute_sentence(data["train"][task_config.col_names[0]]), compute_sentence(data["train"][task_config.col_names[1]])
        U_val, V_val = compute_sentence(data["validation"][task_config.col_names[0]]), compute_sentence(data["validation"][task_config.col_names[1]])
        U_test, V_test = compute_sentence(data["test"][task_config.col_names[0]]), compute_sentence(data["test"][task_config.col_names[1]])

        X_train[id] = np.hstack([U_train, V_train])
        X_val[id] = np.hstack([U_val, V_val])
        X_test[id] = np.hstack([U_test, V_test])

    elif "sentence" in embedding_param:
        id = embedding_param.index("sentence")
        X_train[id] = compute_sentence(data["train"][task_config.col_names[0]])
        X_val[id] = compute_sentence(data["validation"][task_config.col_names[0]])
        X_test[id] = compute_sentence(data["test"][task_config.col_names[0]])

embedding_time = time.time() - start_time

Embedding: Using default time tracking


## Save Embeddings to File

In [15]:
for id in embedding_tracker:

    cache_path = pathlib.Path(f"./cache/{embedding_param[id]}/{task_param}")
    cache_path.mkdir(parents=True, exist_ok=True)

    file_names = ['X_train', 'X_val', 'X_test']
    paths = [pathlib.Path(cache_path / f"{f}_{model_param}.npy") for f in file_names]

    with open(paths[0], 'wb') as X_train_file:
        np.save(X_train_file, X_train[id])
    with open(paths[1], 'wb') as X_val_file:
        np.save(X_val_file, X_val[id])
    with open(paths[2], 'wb') as X_test_file:
        np.save(X_test_file, X_test[id])

In [16]:
X_train = np.concatenate(X_train, axis=1)
X_val = np.concatenate(X_val, axis=1)
X_test = np.concatenate(X_test, axis=1)
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape  : {X_val.shape}")
print(f"X_test shape : {X_test.shape}")

X_train shape: (635, 1536)
X_val shape  : (71, 1536)
X_test shape : (146, 1536)


## Transforming Embeddings

In [17]:
######################
# Choose ids of (Ux, Vx) to alter embeddings
column_ids = [] if task_config.sentence_type == "one" else [0]
######################
if column_ids:
    f = lambda embeddings : add_embeddings(embeddings=embeddings, column_ids=column_ids, embedding_size=768, 
                                           is_UV=[False], is_diff=[True], is_mult=[False], use_abs=[True])

    X_train_computed = f(embeddings=X_train)
    X_val_computed = f(embeddings=X_val)
    X_test_computed = f(embeddings=X_test)
else:
    X_train_computed = X_train
    X_val_computed = X_val
    X_test_computed = X_test
    
print(X_train_computed.shape)
print(X_val_computed.shape)
print(X_test_computed.shape)

(635, 768)
(71, 768)
(146, 768)


# Training loop

In [18]:
input_size = X_train_computed.shape[1]

param_grid = {
    'max_epochs': [50],
    'batch_size': [32],
    'learning_rate': [0.001],
    'category': [task_config.class_type],
    'norm': [False],
    'input_size': [768],
    'layer_size': [768],
    'num_classes': [task_config.num_classes],
    'num_layers': [1],
    'weight_decay': [0.0001],
    'patience': [3],
    'min_delta': [0],
}

# default overrides
param_grid['verbose'] = [True]
param_grid['device'] = [device_name]
param_grid['category'] = [task_config.class_type]
param_grid['num_classes'] = [task_config.num_classes]
 
# Create a list of all combinations of hyperparameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
print(f"{len(all_params)} hyperparameter combinations")

1 hyperparameter combinations


In [22]:
# Create output folder if it doesn't exist
if not os.path.exists('output'):
    os.makedirs('output')

# Setup for logging
console_output_filename = f'./output/{"_".join(embedding_param[0])}_{task_param}_console_output.txt'

with open(console_output_filename, 'a') as logfile:
    logfile.write('\n\nBEGIN TRAINING LOOP\n\n')

# Setup for saving results
results_folder = pathlib.Path(f"../../fast-results/results/{embedding_param[0]}/{task_param}")
results_folder.mkdir(parents=True, exist_ok=True)
save_file_id = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = results_folder / f"val_{save_file_id}_{model_param}.csv"

# different metrics are recorded for classification vs regression tasks
if task_config.class_type in ["BC", "MC"]:
    metric_types = ['mcc', 'f1', 'accuracy']
elif task_config.class_type == "R":
    metric_types = ['pearson', 'spearman']

with open(results_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    header = metric_types + ['num_epochs', 'training time / epoch', 'embedding time', 'training energy / epoch', 'embedding energy'] + list(all_params[0].keys())
    writer.writerow(header)
print(f"saving results to ./{results_file}")
metric_types += ["epoch"]
# Saves best accuracy for progress bar display
display_best = float("-inf")

# Iterate over all combinations of hyperparameters
bar = tqdm(enumerate(all_params), total=len(all_params))
for i, params in bar:
    # Formatting params to display
    print_params = params.copy()
    for param in ['category', 'device']:
        del print_params[param]
        
    # reset torch so that results are consistent
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Initialize the model with current set of hyperparameters
    feed_forward = FeedForward(**params)

    metrics, train_times_per_epoch, energy_per_epoch = feed_forward.fit(X_train_computed,
                                                                        Y_train,
                                                                        X_val_computed,
                                                                        Y_val)
    
    # Log average training time per epoch for current parameter set
    # Note: FFN frequently stops early
    training_time = np.mean(train_times_per_epoch)
    training_energy = np.mean(energy_per_epoch) 
    # Compute energy for embedding generation
    embedding_energy = get_energy(embedding_time, device) # This method effectively just computes energy for a given time

    metric_vals = [metrics[mt] for mt in metric_types]
    
    # displaying results in progress bar
    display_recent = metrics["pearson" if task_config.class_type == "R" else "accuracy"]
    display_best = max(display_best, display_recent)
    bar.set_description(f"Best: {display_best:.5f}, Last: {display_recent:.5f}")

    # Write stats to log file
    with open(console_output_filename, 'a') as logfile:
        logfile.write(f"\n\nTraining with parameters:\n{print_params}")
        logfile.write(f"\nEarly stopped on epoch: {metrics['epoch']}")
        for name, val in zip(metric_types, metric_vals):
            logfile.write(f"\nValidation {name}: {val}")
        logfile.write(f"\nTraining time      : {training_time}") 
        logfile.write(f"\nEmbedding time     : {embedding_time}") 
        logfile.write(f"\nTraining energy    : {training_energy}") 
        logfile.write(f"\nEmbedding energy   : {embedding_energy}") 
    # Write to results csv
    with open(results_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        row = metric_vals + [training_time, embedding_time, training_energy, embedding_energy] + list(params.values())
        writer.writerow(row)

saving results to ./../../fast-results/results/meanpool_single/wnli/val_20240204_175313_DistilRoBERTa.csv


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/50 | Training Loss : 0.68562912940979 | Validation Loss : 0.689026951789856
Accuracy : 0.5633802816901409 | f1 : 0.5633802816901409

Epoch 2/50 | Training Loss : 0.6921358704566956 | Validation Loss : 0.6929911375045776
Accuracy : 0.5633802816901409 | f1 : 0.5633802816901409

Epoch 3/50 | Training Loss : 0.6932677030563354 | Validation Loss : 0.6929928064346313
Accuracy : 0.5633802816901409 | f1 : 0.5633802816901409

Epoch 4/50 | Training Loss : 0.6935994029045105 | Validation Loss : 0.6928735971450806
Accuracy : 0.5633802816901409 | f1 : 0.5633802816901409



In [20]:
results_df = pd.read_csv(results_file)
if task_config.class_type in ["BC", "MC"]:
    print_metric = "accuracy"
elif task_config.class_type == "R":
    print_metric = "pearson"

best = results_df[print_metric].max()
best_row = results_df[results_df[print_metric] == best]

results_df

,mcc,f1,accuracy,num_epochs,training time / epoch,embedding time,training energy / epoch,embedding energy,max_epochs,batch_size,...,norm,input_size,layer_size,num_classes,num_layers,weight_decay,patience,min_delta,verbose,device
0,0.0,0.56338,0.56338,1,0.219094,0.000485,16.432066,0.036371,50,32,...,False,768,768,1,1,0.0001,3,0,True,mps


In [21]:
best_row

,mcc,f1,accuracy,num_epochs,training time / epoch,embedding time,training energy / epoch,embedding energy,max_epochs,batch_size,...,norm,input_size,layer_size,num_classes,num_layers,weight_decay,patience,min_delta,verbose,device
0,0.0,0.56338,0.56338,1,0.219094,0.000485,16.432066,0.036371,50,32,...,False,768,768,1,1,0.0001,3,0,True,mps
